In [ ]:
import pandas as pd 
import numpy as np
import pickle
import matplotlib.pyplot as plt

from scipy.signal import resample
from scipy import signal, stats
from scipy.signal import butter, medfilt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Denoise
def butter_highpass_filter(ecg, cutoff, Fs, order):
    nyq = 0.5 * Fs
    normal_cutoff = cutoff / nyq
    
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    y = signal.filtfilt(b, a, ecg)
    return y

def butter_bandstop_filter(ecg, low, high, Fs, order):
    nyq = 0.5 * Fs
    low = low/ nyq
    high = high/ nyq
    b, a = signal.butter(order, [low, high], btype='bandstop')
    y = signal.filtfilt(b,a,ecg)
    return y

def butter_lowpass_filter(ecg, cutoff, Fs, order):
    nyq = 0.5 * Fs
    normal_cutoff = cutoff / nyq
    
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, ecg)
    return y

def denoise_ecg(amuse_signal, normal_signal, stress_signal, meditation_signal, high, low, sampling_rate, order):
    # high pass filter (0.5)
    bw_amuse = butter_highpass_filter(amuse_signal, high, sampling_rate, order)
    bw_normal = butter_highpass_filter(normal_signal, high, sampling_rate, order)
    bw_stress = butter_highpass_filter(stress_signal, high, sampling_rate, order)
    bw_meditation = butter_highpass_filter(meditation_signal, high, sampling_rate, order)

    # notch filter (57,63) - 50 Hz noise 제거
    notch_amuse = butter_bandstop_filter(bw_amuse, 57, 63, sampling_rate, order)
    notch_normal = butter_bandstop_filter(bw_normal, 57, 63, sampling_rate, order)
    notch_stress = butter_bandstop_filter(bw_stress, 57, 63, sampling_rate, order)
    notch_meditation = butter_bandstop_filter(bw_meditation, 57, 63, sampling_rate, order)

    # notch filter (47, 53) - 60 Hz noise 제거 
    notch_amuse = butter_bandstop_filter(notch_amuse, 47, 53, sampling_rate, order)
    notch_normal = butter_bandstop_filter(notch_normal, 47, 53, sampling_rate, order)
    notch_stress = butter_bandstop_filter(notch_stress, 47, 53, sampling_rate, order)
    notch_meditation = butter_bandstop_filter(notch_meditation, 47, 53, sampling_rate, order)

    # low pass filter
    low_amuse = butter_lowpass_filter(notch_amuse, low, sampling_rate, order)
    low_normal = butter_lowpass_filter(notch_normal, low, sampling_rate, order)
    low_stress = butter_lowpass_filter(notch_stress, low, sampling_rate, order)
    low_meditation = butter_lowpass_filter(notch_meditation, low, sampling_rate, order)
    
    return low_amuse, low_normal, low_stress, low_meditation


# Down, up sampling
def resampling(data, original_freq, new_freq):
    original_samples = len(data)
    new_samples = int(original_samples * (new_freq / original_freq))
    
    downsampled = resample(data, new_samples)
    
    return downsampled


# Delete outlier
def Delet_outlier(data, high_level = None, low_level = None):
    
    high = False if high_level is None else high_level
    low = False if low_level is None else low_level
    
    ex_data = pd.Series(data)
    Q1 = ex_data.quantile(0.25)
    Q3 = ex_data.quantile(0.75)
    IQR = Q3 - Q1 
    
    if high and low:
        dff = ex_data[(ex_data <= Q3+(high*IQR)) & (ex_data >= Q1-(low*IQR))]
        
    elif high and not low:
        dff = ex_data[(ex_data <=Q3+(high*IQR))]
        
    elif not high and low:
        dff = ex_data[(ex_data >= Q1-(low*IQR))]
    else :
        dff = ex_data
    
    #dff = ex_data[(ex_data <= Q3+(high_level*IQR)) & (ex_data >= Q1-(low_level*IQR))]
    
    dff = dff.reset_index(drop=True)
    
    return np.array(dff)

# minmax normalization (0,1)
def minmax(data):
    scaler = MinMaxScaler()
    new = scaler.fit_transform(data.reshape(-1,1))
    return new

# plot
def plot_ecg(amuse, normal, stress, meditation):
    plt.figure(figsize=(20,3))
    plt.subplot(1,4,1)
    plt.plot(amuse); plt.title('Amuse')
    plt.subplot(1,4,2)
    plt.plot(normal); plt.title('Normal')
    plt.subplot(1,4,3)
    plt.plot(stress); plt.title('Stress')
    plt.subplot(1,4,4)
    plt.plot(meditation); plt.title('meditation')
    plt.show()
    

# Initial Settings

In [ ]:
Data = "D:/Database/WESAD/"
save_path = "D:/Journal/WESAD_Denoise_Outlier_Normalization/"

WESAD_freq = 700
StressID_freq = 700

# filtering
high_cutoff = 0.5
sampling_rate = 700
ts = 1 / sampling_rate
nyq = 0.5 * sampling_rate # 나이퀴스트 
order = 4
low_cutoff = 100

## Process
### 1. Outlier
### 2. Denoise
### 3. Normalization

# S2
- amuse, stress 이상치 X

In [ ]:
df = pd.read_pickle(Data + 'S2' + '/' + 'S2' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

In [ ]:
out_normal = Delet_outlier(normal_ecg, 5, 2)
out_meditation = Delet_outlier(meditation_ecg, 13, 0)
plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(amuse_ecg, out_normal, stress_ecg, out_meditation)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, out_normal, stress_ecg, out_meditation, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, out_normal, stress_ecg, out_meditation)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S2.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S3

In [ ]:
df = pd.read_pickle(Data + 'S3' + '/' + 'S3' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

In [ ]:
out_stress = Delet_outlier(stress_ecg, 11, 4)
plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(amuse_ecg, out_normal, out_stress, meditation_ecg)

In [ ]:
plt.plot(meditation_ecg[700:1000])
plt.plot(de_meditation[700:1000])

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, out_stress, meditation_ecg, 0.5, 100, 700, 4)
de_meditation = de_meditation[700:]
plot_ecg(amuse_ecg, normal_ecg, out_stress, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S3.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S4
이상치 없음

In [ ]:
df = pd.read_pickle(Data + 'S4' + '/' + 'S4' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

In [ ]:
plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S4.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S5
Stress 이상치 제거

In [ ]:
df = pd.read_pickle(Data + 'S5' + '/' + 'S5' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
out_stress = Delet_outlier(stress_ecg, None, 8)
plot_ecg(amuse_ecg, normal_ecg, out_stress, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, out_stress, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, normal_ecg, out_stress, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S5.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S5.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S6
이상치 없음

In [ ]:
df = pd.read_pickle(Data + 'S6' + '/' + 'S6' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S6.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S6.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S7
이상치 없음

In [ ]:
df = pd.read_pickle(Data + 'S7' + '/' + 'S7' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S7.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S7.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S8
정상, 스트레스 이상치 존재

In [ ]:
df = pd.read_pickle(Data + 'S8' + '/' + 'S8' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
out_normal = Delet_outlier(normal_ecg, None, 14)
out_stress = Delet_outlier(stress_ecg, None, 2.5)

plot_ecg(amuse_ecg, out_normal, out_stress, meditation_ecg)

In [ ]:
plt.plot(meditation_ecg[0:2000])
plt.plot(de_meditation[0:2000])
plt.show()
plt.plot(meditation_ecg[0:2000])
plt.plot(de_meditation[0:2000])

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, out_normal, out_stress, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, out_normal, out_stress, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
out_de_meditation = Delet_outlier(de_meditation, 22, 9)
plt.plot(de_meditation)
plt.plot(out_de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(out_de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S8.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S8.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S9

In [ ]:
df = pd.read_pickle(Data + 'S9' + '/' + 'S9' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
re_meditation_ecg = meditation[350:]

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(re_meditation_ecg)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S9.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S9.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S10

In [ ]:
df = pd.read_pickle(Data + 'S10' + '/' + 'S10' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
out_amuse = Delet_outlier(amuse_ecg, 10, 6)
out_normal = Delet_outlier(normal_ecg, 10, 7)
out_stress = Delet_outlier(stress_ecg, None, 8)
plot_ecg(out_amuse, out_normal, out_stress, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(out_amuse, out_normal, out_stress, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(out_amuse, out_normal, out_stress, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
de_out_amuse = Delet_outlier(de_amuse, 10, 6)
plt.plot(de_amuse)
plt.plot(de_out_amuse)

In [ ]:
de_out_normal = Delet_outlier(de_normal, 10, 7)
plt.plot(de_normal)
plt.plot(de_out_normal)

In [ ]:
amuse = minmax(de_out_amuse)
normal = minmax(de_out_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S10.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S10.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S11
이상치 없음

In [ ]:
df = pd.read_pickle(Data + 'S11' + '/' + 'S11' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S11.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S11.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S13

In [ ]:
df = pd.read_pickle(Data + 'S13' + '/' + 'S13' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S13.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S13.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S14

In [ ]:
df = pd.read_pickle(Data + 'S14' + '/' + 'S14' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
plt.plot(normal_ecg[0:1000])
plt.plot(de_normal[0:1000])

In [ ]:
re_de_normal_ecg = de_normal[400:]

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(re_de_normal_ecg)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S14.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S14.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S15

In [ ]:
df = pd.read_pickle(Data + 'S15' + '/' + 'S15' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
out_normal = Delet_outlier(normal_ecg, None, 4)
out_stress = Delet_outlier(stress_ecg, None, 4)
plot_ecg(amuse_ecg, out_normal, out_stress, meditation_ecg)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, out_normal, out_stress, meditation_ecg, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, out_normal, out_stress, meditation_ecg)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S15.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S15.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S16

In [ ]:
df = pd.read_pickle(Data + 'S16' + '/' + 'S16' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
out_meditation = Delet_outlier(meditation_ecg, None, 4)
plot_ecg(amuse_ecg, normal_ecg, stress_ecg, out_meditation)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(amuse_ecg, normal_ecg, stress_ecg, out_meditation, 0.5, 100, 700, 4)

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, out_meditation)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
out_meditation = Delet_outlier(meditation_ecg, None, 4)
plot_ecg(de_amuse, de_normal, de_stress, out_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(out_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

name = 'S16'

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S16.pkl','wb') as fw:
    pickle.dump(new_data, fw)

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S16.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# S17

In [ ]:
df = pd.read_pickle(Data + 'S17' + '/' + 'S17' + '.pkl')
ecg_v = df['signal']['chest']['ECG']
label = df['label']

amuse_ecg = ecg_v[np.where(label == 3)[0]].flatten()
normal_ecg = ecg_v[np.where(label == 1)[0]].flatten()
stress_ecg = ecg_v[np.where(label == 2)[0]].flatten()
meditation_ecg = ecg_v[np.where(label == 4)[0]].flatten()

plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)

In [ ]:
out_amuse = Delet_outlier(amuse_ecg, None, 5)
out_meditation = Delet_outlier(meditation_ecg, None, 3)
plot_ecg(amuse_ecg, normal_ecg, stress_ecg, meditation_ecg)
plot_ecg(out_amuse, normal_ecg, stress_ecg, out_meditation)

In [ ]:
de_amuse, de_normal, de_stress, de_meditation = denoise_ecg(out_amuse, normal_ecg, stress_ecg, out_meditation, 0.5, 100, 700, 4)

plot_ecg(out_amuse, normal_ecg, stress_ecg, out_meditation)
plot_ecg(de_amuse, de_normal, de_stress, de_meditation)

In [ ]:
amuse = minmax(de_amuse)
normal = minmax(de_normal)
stress = minmax(de_stress)
meditation = minmax(de_meditation)

print(amuse.shape, normal.shape, stress.shape, meditation.shape)
plot_ecg(amuse, normal, stress, meditation)
plot_ecg(amuse[20000:30000], normal[20000:30000], stress[20000:30000], meditation[20000:30000])

# 3 Class

In [ ]:
amuse_d = resampling(amuse ,WESAD_freq, StressID_freq)
normal_d = resampling(normal ,WESAD_freq, StressID_freq)
stress_d = resampling(stress ,WESAD_freq, StressID_freq)

plot_ecg(amuse_d, normal_d, stress_d)

new_data = {'excitement' : amuse_d, 'neutral' : normal_d, 'stress' : stress_d}

with open(save_path + 'S17.pkl','wb') as fw:
    pickle.dump(new_data, fw)

# 4 Class

In [ ]:
new_data = {'excitement' : amuse, 'baseline' : normal, 'stress' : stress, 'meditation' : meditation}

with open(save_path + 'S17.pkl','wb') as fw:
    pickle.dump(new_data, fw)